In [56]:
name="小王"
month="三月"
print("{}在{}的工资为{}".format(name, month,5000))

小王在三月的工资为5000


In [61]:
import torchomport torchnvjpeg
import torch.nn.functional as F
def load_model(model_path, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = torch.load(model_path, map_location=device)
    return model.to(device)
def prepare_input(device, batch_size=1, channels=3, height=640, width=640):
    x = torch.randn(batch_size, channels, height, width, device=device)
    return x
def infer(model_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 加载模型
    model = load_model(model_path, device)
    model.eval()  # 设置为评估模式
    
    # 准备输入数据
    x = prepare_input(device)
    
    # 执行前向传播
    with torch.no_grad():
        output = model(x)
    
    # 打印输出的形状（作为示例）
    print(output.size())

# 调用推理函数
model_path = 'best.pt'  # 确保这是你的模型文件的正确路径
infer(model_path)


AttributeError: 'dict' object has no attribute 'to'

In [63]:
import cv2

for i in range(100):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print('Webcam found at index: ' + str(i))
        cap.release()

Webcam found at index: 0
Webcam found at index: 1
